# EasyLLM 품질 데이터 필터 사용 방법

EasyLLM의 `data` 패키지는 향상된 사전 학습을 위해 텍스트 데이터를 전처리하기 위한 품질 필터를 추가합니다.

In [ ]:
!pip install "easyllm[data]" --upgrade

## 1. 혼란도 필터링

혼란도 필터링은 혼란스러운 텍스트 세그먼트를 제거하고 모델 학습을 보다 표준적이고 이해하기 쉬운 언어에 집중함으로써 모델 품질, 일관성 및 학습 효율성을 향상시키는 데 사용할 수 있습니다.
혼란도 필터링은 위키백과에서 학습된 `KenLM` 모델을 사용하여 구현됩니다. 언어 ID(예: `de`)와 혼란도 `min_threshold` 및 `max_threshold`를 제공하기만 하면 필터는 텍스트의 혼란도가 임계값 `False`를 벗어나면 `True`를 반환합니다.


In [2]:
from easyllm.data.filters import PerplexityFilter

ppl = PerplexityFilter("en",min_threshold=10,max_threshold=1000)

# 혼란도 가져오기
print(ppl.model.get_perplexity("나는 매우 당혹스럽다"))
# 341.3 (문장 스타일이 형식적이고 문법 오류가 없으므로 혼란도가 낮음)

print(ppl.model.get_perplexity("나는 완전히 미쳤다"))
# 46793.5 (문장이 구어체이고 문법 오류가 포함되어 있으므로 혼란도가 높음)

# 필터 테스트
assert ppl("나는 매우 당혹스럽다") == False


341.3
46793.5


## NonAlphaNumericFilter

`NonAlphaNumericFilter`는 문서의 영숫자가 아닌 문자 수를 기준으로 문서를 제거합니다. [Gopher (Rae et al., 2021)](https://arxiv.org/pdf/2112.11446.pdf)을 기반으로 하며, 문서에 영숫자가 아닌 문자가 20% 이상 포함되어 있으면 제거됩니다.

In [1]:
from easyllm.data.filters import NonAlphaNumericFilter

nam = NonAlphaNumericFilter()

# 필터링되지 않음
assert nam("이것은 테스트입니다") == False

# 필터링됨
assert nam("이것은 테스트입니다!!!!!!!") == True


## SymbolToWordFilter

`SymbolToWordFilter`는 해시 기호 또는 줄임표에 대한 기호 대 단어 비율이 0.1보다 큰 문서를 제거합니다. [Gopher (Rae et al., 2021)](https://arxiv.org/pdf/2112.11446.pdf) 기반

In [1]:
from easyllm.data.filters import SymbolToWordFilter

stw = SymbolToWordFilter()

assert stw("이것은 테스트입니다") == False

assert stw("스팸#스팸#스팸#스팸#스팸#스팸#스팸#스팸") == True

## NumbersToCharacterFilter

`NumbersToCharacterFilter`는 문서의 20%가 숫자인 문서를 제거합니다.

In [1]:
from easyllm.data.filters import DigitToCharacter

ntw = DigitToCharacter()

assert ntw("안녕하세요 123 세상 456 이 텍스트 789에는 1234 숫자가 더 많은 단어가 포함되어 있습니다") == False

assert ntw("안녕하세요 34534 34534 ") == True


## UrlRatioFilter

`UrlRatioFilter`는 문서의 20%가 URL인 문서를 제거합니다.

In [2]:
from easyllm.data.filters import UrlRatioFilter 

ur = UrlRatioFilter()

assert ur("https://www.google.com") == True

assert ur("일부 URL http://www.example.com이 포함된 예제 텍스트와 더 많은 텍스트 https://www.example2.com 및 더 많은 텍스트") == False

## BulletpointRatioFilter 

`BulletpointRatioFilter`는 글머리 기호가 90% 이상인 문서를 제거합니다. [Gopher (Rae et al., 2021)](https://arxiv.org/pdf/2112.11446.pdf) 기반

In [1]:
from easyllm.data.filters import BulletpointRatioFilter

br = BulletpointRatioFilter()

assert br("이것은 \n- 일부 글머리 기호가 있지만\n모두는 아닌 텍스트입니다") == False

assert br("- 일부 글머리 기호 및\n- 더 많은 글머리 기호") == True

## WhitespaceRatioFilter

`WhitespaceRatioFilter`는 텍스트의 25% 이상이 공백인 문서를 제거하는 필터입니다.


In [2]:
from easyllm.data.filters import WhitespaceRatioFilter

wr = WhitespaceRatioFilter()

assert wr("이것은 테스트입니다") == False

assert wr("안녕하세요 세상!      이 텍스트에는    추가 공백이 있습니다.") == True

## ParenthesesRationFilter

`ParenthesesRationFilter`는 괄호 비율이 10%보다 큰 모든 문장을 제거하는 필터입니다.

In [1]:
from easyllm.data.filters import ParenthesesRationFilter

pr = ParenthesesRationFilter()

assert pr("이것은 일반적인 문장입니다") == False

assert pr("이것은 (괄호 ) ] {(가 있는) 문장입니다") == True

## LongWordFilter

`LongWordFilter`는 1000자보다 긴 단어(예: js 축소 파일)를 포함하는 문서를 제거하는 필터입니다.

In [2]:
from easyllm.data.filters import LongWordFilter

lw = LongWordFilter()

assert lw("이것은 테스트입니다") == False

assert lw(f"이것은 {'긴단어'*500}가 있는 테스트입니다") == True

## LengthFilter

`LengthFilter`는 특정 단어 수보다 작거나 큰 문서를 제거합니다. 계산 비용이 더 많이 들기 때문에 토큰이 아닙니다.

In [1]:
from easyllm.data.filters import LengthFilter

l = LengthFilter(min_length=1, max_length=100)

assert l("안녕하세요 세상") == False

assert l("안녕하세요 세상 " * 100) == True

## RepeatedParagraphFilter, RepeatedLinesFilter

`RepeatedParagraphFilter` 및 `RepeatedLinesFilter`는 반복되는 줄이나 단락이 30% 이상인 문서를 제거합니다. [Gopher (Rae et al., 2021)](https://arxiv.org/pdf/2112.11446.pdf) 기반

In [1]:
from easyllm.data.filters import RepeatedLinesFilter, RepeatedParagraphFilter

rl = RepeatedLinesFilter()
rp = RepeatedParagraphFilter()

assert rl("안녕하세요\n세상") == False
assert rl("안녕하세요\n안녕하세요\n안녕하세요\n안녕하세요") == True

assert rp("안녕하세요\n\n세상") == False
assert rp("안녕하세요\n\n안녕하세요\n\n안녕하세요\n\n안녕하세요") == True

## TopNGramsFilter

`TopNGramsFilter`는 상위 n-gram이 문서의 20% 이상을 차지하는 경우 문서를 제거합니다.

In [1]:
from easyllm.data.filters import TopNGramsFilter

tng = TopNGramsFilter()

assert tng("이것은 더 긴 문장에 대한 테스트입니다") == False 

assert tng("빠른 갈색 여우가 게으른 개를 뛰어넘습니다 빠른 갈색") == True

## PunctuationFilter & EllipsisFilter

`PunctuationFilter` 및 `EllipsisFilter`는 "줄 바꿈"의 15% 이상에 구두점이 없거나 "줄 바꿈"의 30% 이상에 줄임표가 포함된 경우 문서를 제거합니다.

In [3]:
from easyllm.data.filters import PunctuationFilter, EllipsisFilter

pf = PunctuationFilter()

assert pf("이것은 문장입니다.") == False

assert pf("이것은 문장입니다\n 하지만 그렇지 않습니다.\n아직 아무도 없습니다.") == True

ef = EllipsisFilter()

assert ef("이것은 문장입니다.") == False

assert ef("이것은 문장입니다\n 하지만 그렇지 않습니다....") == True

## CommonWordFilter

`CommonWordFilter`는 일반적인 단어가 2개 이상 포함되어 있지 않으면 문서를 제거합니다.

In [1]:
from easyllm.data.filters import CommonWordFilter

cw = CommonWordFilter()

assert cw("이것은 일반적인 단어가 포함된 문장입니다.") == False

assert cw("고양이 개 쥐") == True